This is the code to run benchmarking tests of the **STANDARD PRM - PARALLEL** implementation algorithm.
The same simulated 2-d configuration space for all four benchmarking tests is included here.  

To run this code, please run the below three boxes to set-up the simulation. 

The last box runs the simulation, you may alter the NUM_SAMPLES and RUN keyword arguments in the prm() method call. 
NUM_SAMPLES specifies the number of samples that should be generated. 
RUN specifies the number of trials that should be ran. The output results are the average runtime across RUN number of trials.

In [2]:
!pip install pycuda # install cuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.6 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2022.2.2-cp310-cp310-linux_x86_64.whl size=661989 sha256=9f44c16553017cc0a1f7c62ed9bde71cb81e032cea3995064c5dcf0e51ebbd27
  Stored in directory: /root/.cache/pip/wheels/1d/7b/06/82a395a243fce00035dea9914d92bbef0013401497d849f8bc
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69866 sha256=e7ffcab2ee0d4e5284c6e2ea5542203081a608eff6038d1dee39ef0894913fbe
  Stored in directory: /root/.cache/pip/wheels/19/02

In [1]:
import pycuda.driver as cuda
import pycuda.curandom as curandom
import pycuda.autoinit
from pycuda.compiler import SourceModule
import cv2
from google.colab.patches import cv2_imshow
from google.colab import output
import time 
import os, sys
# set SDL to use the dummy NULL video driver, 
  # so it doesn't need a windowing system.
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [2]:
%%writefile prm4.py
# prm.py
# This program runs the vanilla PRM algorithm
#  
# Authors: Kevin Grant Li, Avigayil Helman, Kevin Li, Joseph Han,  Nicole Neil
# 
# Adapted from code from Professor Brian Plancher and Avigayil Helman
# Adapted from code written by Steve LaValle and Rus Tedrake

import sys, random, math, pygame
from pygame.locals import *
from util import Util
import heapq
from decimal import Decimal, getcontext
import timeit
import time
from itertools import product
import numpy
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import math

class prm:
    def __init__(self, obstacles, start_node, goal_node, XDIM = 640, YDIM = 480, NUM_SAMPLES = 0, LINE_WIDTH = 20, TEST_MODE = 0, K = 5, RUNS = 1):
        
        self.obstacles = obstacles # Obstacles are represented by a list of points defining line segments 
        self.LINE_WIDTH = LINE_WIDTH # width of obstacle lines (projected out)
        self.XDIM = XDIM # board dimmension -> x-dimension 
        self.YDIM = YDIM # board dimmension -> y-dimension
        self.start_node = start_node # (x,y) position of starting location  
        self.goal_node = goal_node # (x,y) position of goal location 
        self.NUM_SAMPLES = NUM_SAMPLES # total number of samples we will make within the plane 
        self.K = K # number of nearest neighbors for building out graph
        self.RUNS = RUNS
        self.context = cuda.Device(0).make_context()

    def runGame_benchmark(self):

        total_time = 0
        sample_total_time = 0
        knn_total_time = 0
        build_edge_total_time = 0
        check_edge_total_time = 0
        search_total_time = 0

        print("starting test")
        for i in range(self.RUNS):
            ########################
            ## INNER runGame Func ##
            ########################

            # start the list of nodes --> make sure to include both the start state and the end state within this list!
            nodes = set()
            parents = {}
            wastes = 0

            new_obstacles = self.redefineObstacles(self.obstacles, self.LINE_WIDTH)

            overall_start_time = timeit.default_timer()

            #grid_sampling_start_time = timeit.default_timer()
            grid_samples = self.sample_grid_parallel({self.start_node, self.goal_node}, self.XDIM, self.YDIM, self.NUM_SAMPLES) 
            #grid_sampling_time.append(timeit.default_timer() - grid_sampling_start_time)
            
            #remove_obstructed_start_time = timeit.default_timer()
            non_obstructed_samples = self.remove_obstacle_samples_parallel(grid_samples,self.obstacles,self.LINE_WIDTH,1)
            #remove_obstructed_time.append(timeit.default_timer() - remove_obstructed_start_time)

            sample_total_time += timeit.default_timer() - overall_start_time

            nodes = self.buildNodes(non_obstructed_samples, self.K)


            start_time = timeit.default_timer()
            self.identifyNearestNeighbors_parallel(nodes, self.K)
            knn_total_time += timeit.default_timer() - start_time

            start_time = timeit.default_timer()
            edges = self.buildEdges(nodes)
            build_edge_total_time += timeit.default_timer() - start_time

            start_time = timeit.default_timer()
            unobstructedEdges = self.removeObstructedEdges_parallel(edges, new_obstacles, self.LINE_WIDTH)
            check_edge_total_time += timeit.default_timer() - start_time

            for node in nodes:
                if node.access_xy() == self.start_node:
                    starting_node = node
                elif node.access_xy() == self.goal_node:
                    endgoal_node = node

            start_time = timeit.default_timer()
            shortest_path = self.dijkstra(starting_node, endgoal_node, unobstructedEdges)
            search_total_time += timeit.default_timer() - start_time

            ########################
            ########  END  #########
            ########################

        n = self.RUNS
        print("total avg time: ", (sample_total_time + knn_total_time + build_edge_total_time + check_edge_total_time + search_total_time)/n)
        print("sampling avg time: ", sample_total_time / n)
        print("knn avg time: ", knn_total_time / n)
        print("check edge collision avg time: ", check_edge_total_time / n)
        print("dijkstra's avg time: ", search_total_time / n)
        self.context.pop()


    # samples grid generation parallel version
    def sample_grid_parallel(self, nodes, XDIM, YDIM, NUM_SAMPLES):
      # # CUDA kernel to generate random samples

      mod = SourceModule("""
      #include <curand_kernel.h>

      extern "C"{
          __global__ void test_kernel(float* contents, int XDIM, int YDIM)
          {
              int tId = threadIdx.x + (blockIdx.x * blockDim.x);
              curandState state;
              curand_init((unsigned long long)clock() + tId, 0, 0, &state);

              float rand1 = (float) curand_uniform_double(&state);

              curand_init((unsigned long long)clock() + tId, 0, 0, &state);
              float rand2 = (float) curand_uniform_double(&state);


              contents[tId * 2] = rand1 * XDIM; 
              contents[tId * 2 + 1] = rand2 * YDIM; 
          }
      }
      """, no_extern_c=True)

      nodes = numpy.array(list(nodes), dtype=numpy.float32)

      out_cpu = numpy.zeros((NUM_SAMPLES - 2) * 2, dtype=numpy.float32)
      out_gpu = cuda.mem_alloc((NUM_SAMPLES - 2) * 2 * numpy.dtype(numpy.float32).itemsize)


      start_time = timeit.default_timer()
      func = mod.get_function("test_kernel")
      block_size = 1024
      grid_size = ((NUM_SAMPLES - 2) + block_size - 1) // block_size
      func(out_gpu, numpy.int32(XDIM), numpy.int32(YDIM), block=(block_size, 1, 1), grid=(grid_size, 1, 1))

      #cuda.Device(0).synchronize()
      cuda.memcpy_dtoh(out_cpu, out_gpu)
      #cuda.free(out_gpu)
      out_gpu.free()

      start_time = timeit.default_timer()
      samples_np = numpy.reshape(out_cpu, (-1, 2))
      samples_np = numpy.concatenate((nodes, samples_np), axis=0)
      samples_np = set(tuple(row) for row in samples_np)
      return samples_np

    def remove_obstacle_samples_parallel(self,grid_samples,obstacles,obstacle_width,u): 

        # Define the CUDA Kernel 
        mod = SourceModule("""

        #include <cmath>

        __device__ int distance(int ax, int ay, int bx, int by)
        {
            float dx = ax - bx; 
            float dy = ay - by; 
            float d = std::sqrt(dx * dx + dy * dy); 
            return d; 
        }


        __global__ void remove_obstructed_samples(int* grid_samples, int* obstacles, int* no_obstacles_a, int* flags, int* obstacle_width_a,int* sample_num)
        { 
            int no_obstacles = no_obstacles_a[0]; 
            int obstacle_width = obstacle_width_a[0]; 

            int sample_no = threadIdx.x + blockIdx.x * blockDim.x; 

            if (sample_no>sample_num[0])return;

            int pointx = grid_samples[sample_no * 2]; 
            int pointy = grid_samples[sample_no * 2 + 1];   

            float half_width = (obstacle_width/2); 
            float sq_w = half_width * half_width; 

            for(int i = 0; i < no_obstacles; i++){
            
                int obstacle_x1 = obstacles[i * 4]; 
                int obstacle_y1 = obstacles[i * 4 + 1]; 
                int obstacle_x2 = obstacles[i * 4 + 2]; 
                int obstacle_y2 = obstacles[i * 4 + 3]; 

                float c = distance(obstacle_x1, obstacle_y1, obstacle_x2, obstacle_y2); 
                float a = distance(obstacle_x1, obstacle_y1, pointx, pointy); 
                float b = distance(obstacle_x2, obstacle_y2, pointx, pointy); 

                if (half_width >= a || half_width >= b){
                
                    if (a < b){
                        sq_w = a * a; 
                    } 
                    else {
                        sq_w = b * b; 
                    }
                }  

                float c1 = std::sqrt(a * a - sq_w); 
                float c2 = std::sqrt(b * b - sq_w); 

                if (c1 + c2 <= c){
                    flags[sample_no] = 1;  
                }
            }      
        }
        """)
        
        # create a grid samples array -> [x1, y1, x2, y2, x3, y3 .... ]
        host_grid_samples = numpy.array(list(grid_samples)).flatten().astype(numpy.int32)
        device_grid_samples_array = cuda.mem_alloc(host_grid_samples.nbytes)
        cuda.memcpy_htod(device_grid_samples_array, host_grid_samples)

        # create a obstacles array -> [xa1, ya1, xa2, ya2, xb1, yb1, xb2, yb2 ... ]

        host_obstacles = numpy.array(list(obstacles.values())).flatten().astype(numpy.int32)
        device_obstacles_array = cuda.mem_alloc(host_obstacles.nbytes)
        cuda.memcpy_htod(device_obstacles_array, host_obstacles)

        # flag for each grid samples, 0 if not obstructed, 1 if obstructed
        host_obstructed_flag = numpy.zeros(len(grid_samples),dtype=numpy.int32)
        device_obstructed_flag = cuda.mem_alloc(host_obstructed_flag.nbytes)
        cuda.memcpy_htod(device_obstructed_flag, host_obstructed_flag)

        # to pass in the other information need to construct the relevant array 
        no_obstacles = numpy.array([len(obstacles)], dtype=numpy.int32)
        no_obstacles_a = cuda.to_device(no_obstacles)

        obstacle_width_a = cuda.mem_alloc(4)
        cuda.memcpy_htod(obstacle_width_a, numpy.array([obstacle_width], dtype=numpy.int32))    

        sample_num = cuda.mem_alloc(4)
        cuda.memcpy_htod(sample_num, numpy.array([len(grid_samples)], dtype=numpy.int32))      

        # call the CUDA kernel
        func = mod.get_function("remove_obstructed_samples")

        # UPDATED
        block_size = 1024
        grid_size = (len(grid_samples) + block_size - 1) // block_size
        func(device_grid_samples_array, device_obstacles_array, no_obstacles_a, device_obstructed_flag, obstacle_width_a, sample_num,block=(block_size, 1, 1), grid=(grid_size, 1, 1))
        #cuda.Device(0).synchronize()

        # copy back the device memory to the host memory
        cuda.memcpy_dtoh(host_obstructed_flag, device_obstructed_flag) 

        # Free Used Memory
        #cuda.free(device_grid_samples_array)
        #cuda.free(device_obstacles_array)
        #cuda.free(device_obstructed_flag)
        #cuda.free(obstacle_width_a)
        #cuda.free(sample_num)

        device_grid_samples_array.free()
        device_obstacles_array.free()
        device_obstructed_flag.free()
        obstacle_width_a.free()
        sample_num.free()


        # select out only the samples where flag is 0 (ie not colliding)
        unobstructed_samples = [tup for i, tup in enumerate(list(grid_samples)) if host_obstructed_flag[i] == 0]

        return unobstructed_samples

    # builds all nodes from a list of (x,y) tuples
    def buildNodes(self,samples,K):
        nodes = [] 
        for sample in samples: 
            nodes.append(Node(sample,K))
        return nodes 

    # updates each node with its k-nearest neighbors
    def identifyNearestNeighbors_parallel(self, nodes, K): 
        # Parallel KNN
        self.parallel_identifyNearestNeighbors(nodes, K)

    # Algorithm: pre-store the distance between the current node and the first K nodes in a distance array of K elements, and initialize a "winner" array of indices to 0 to K-1.
    # Both the distance array and the winner array of indices will be updated as we go along and compare distances
    # Iterate through distances between current node and all nodes. When a pair of nodes produce a smaller distance than the largest distance in the distance array, swap them and their indices in the winner array, and sort them
    # When the iterations are complete, the winner array will end up with the indices of the K nodes with the shortest distances with the current node
    # For parallelization, launch all 1024 threads and subdivide the total number of nodes by the number of threads
    def parallel_identifyNearestNeighbors(self, nodes, K):
        length = len(nodes)
        m = K
        # Data conversion for passing into CUDA kernel
        nodesList = numpy.array(list(map(lambda x: [x.access_xy()[0], x.access_xy()[1]], nodes)))

        distList = numpy.tile(range(m), (length, 1))
        indList = numpy.tile(range(m), (length, 1))
        cartesian_product = list(product(range(length), range(m))) # Calculate initial K distances for use in kernel function
        distList = numpy.array(list(map(lambda x: (nodesList[x[0]][0] - nodesList[x[1]][0])**2 + (nodesList[x[0]][1] - nodesList[x[1]][1])**2 if x[0] != x[1] else 1E38, cartesian_product)))
        distList = distList.reshape((length, m))

        nodesList = nodesList.astype(numpy.float32)
        distList = distList.astype(numpy.float32)
        indList = indList.astype(numpy.int32)

        numNodes = numpy.int32(length)
        topM = numpy.int32(m)
        numthreads = int(1024)
        threadSize = numpy.int32(numpy.ceil(numNodes/numthreads))

        # Memory allocation and movement of data
        nodesList_gpu = cuda.mem_alloc(nodesList.nbytes)
        distList_gpu = cuda.mem_alloc(distList.nbytes)
        indList_gpu = cuda.mem_alloc(indList.nbytes)
        cuda.memcpy_htod(nodesList_gpu, nodesList)
        cuda.memcpy_htod(distList_gpu, distList)
        cuda.memcpy_htod(indList_gpu, indList)

        # Kernel function
        mod = SourceModule("""
          __device__ void swap_data(float *a, float *b) {
            float temp = *a;
            *a = *b;
            *b = temp;
          }

          __device__ void swap_index(int *a, int *b) {
            int temp = *a;
            *a = *b;
            *b = temp;
          }

          __device__ void bubble_sort(float *list, int *indlist, int m)
          {
              bool swapped;
              for (int i = 0; i < m-1; i++)
              {
                  swapped = false;
                  for (int j = 0; j < m-i-1; j++)
                  {
                      if (list[j] > list[j+1])
                      {
                          swap_data(&list[j], &list[j+1]);
                          swap_index(&indlist[j], &indlist[j+1]);
                          swapped = true;
                      }
                  }

                  if (!swapped) {
                      break;
                  }
              }
          }

          // Distance function
          __device__ float dist(float x1, float y1, float x2, float y2) {
            return (x2-x1)*(x2-x1) + (y2-y1)*(y2-y1);
          }

          // Iterate through all nodes and compare with the current node. If distance is smaller than the greatest of the m current "winners", swap them and sort the new top m winners
          __device__ void topM(float *nodesList, float *distList, int *indlist, float x, float y, int m, int numNodes) {
            bubble_sort(distList, indlist, m);
            for (int i = m; i < numNodes; i++) {
              int index = i;
              float distance = dist(nodesList[2*i], nodesList[2*i+1], x, y);
              if (distance < 1E-34) continue;
              if (distance < distList[m - 1]) {
                swap_data(&distance, &distList[m - 1]);
                swap_index(&index, &indlist[m - 1]);
                bubble_sort(distList, indlist, m);
              }
            }
          }

          // KNN for one particular node
          // nodesList: comes in pairs of x and y coordinates, of length numNodes*2
          // distsList: stores the m min distances for a particular node, of length m. Initialized to distances between this particular node and the first m nodes (FIXME: note exceptions)
          // indList: stores the m node indices to connect to for min distance, of length m. Initialized to the first m indices (FIXME: note exceptions)
          // x, y: x and y coordinates of particular node
          // m: number of k nearest neighbors
          // numNodes: number of nodes to check through
          __device__ void parallel(float *nodesList, float *distList, int *indList, float x, float y, int m, int numNodes) {
            topM(nodesList, distList, indList, x, y, m, numNodes);
          }

          // KNN for all nodes in threadblock
          __global__ void knn(float *nodesList, float *distList, int *indList, int m, int numNodes, int threadSize)
          {
            int idx;
            int idy;
            int index;

            for (int i = 0; i < threadSize; i++) {
              idx = (threadIdx.x*threadSize + i) * 2;
              idy = (threadIdx.x*threadSize + i) * 2 + 1;
              // nodesList[idx] = i;
              // nodesList[idy] = -i;
              index = (threadIdx.x*threadSize+i)*m;
              parallel(nodesList, distList+index, indList+index, nodesList[idx], nodesList[idy], m, numNodes);
            }
          }
          """)
        
        func = mod.get_function("knn")
        func(nodesList_gpu, distList_gpu, indList_gpu, topM, numNodes, threadSize, block=(numthreads,1,1))
        # cuda.Device(0).synchronize()

        a_doubled = numpy.empty_like(nodesList)
        cuda.memcpy_dtoh(a_doubled, nodesList_gpu)
        b_doubled = numpy.empty_like(distList)
        cuda.memcpy_dtoh(b_doubled, distList_gpu)
        c_doubled = numpy.empty_like(indList)
        cuda.memcpy_dtoh(c_doubled, indList_gpu)

        #cuda.free(nodesList_gpu)
        #cuda.free(distList_gpu)
        #cuda.free(indList_gpu)
        nodesList_gpu.free()
        distList_gpu.free()
        indList_gpu.free()

        for i in range(length):
            nodes[i].set_nearest_neighbors([nodes[j] for j in c_doubled[i]])


    # builds all edges -> edges are connections from a node to its k-nearest neighbors
    def buildEdges(self,nodes): 
        edges = list()

        for node1 in nodes: 
            for node2 in node1.k_nearest_neighbors: 
                potential_edge = Edge(node1,node2)
                edges.append(potential_edge)

        return edges
  

    # redefine obstacles as a list of rects
    def redefineObstacles(self, obstacles, obstacle_width):
        new_obstacles = []
        for key in obstacles.keys():
            obstacle = obstacles[key]
            for i in range(len(obstacle)-1):
                j = i + 1

                rect_points = [None] * 4

                if (obstacle[i][0] == obstacle[j][0]):
                    x1 = obstacle[i][0] - obstacle_width / 2
                    x2 = obstacle[i][0] + obstacle_width / 2
                    y1 = obstacle[i][1]
                    y2 = obstacle[j][1]

                    if y1 > y2:
                        y1, y2 = y2, y1

                    rect_points[0] = (x1, y1)
                    rect_points[1] = (x2, y1)
                    rect_points[2] = (x2, y2)
                    rect_points[3] = (x1, y2)

                elif (obstacle[i][1] == obstacle[j][1]):
                    x1 = obstacle[i][0]
                    x2 = obstacle[j][0]
                    y1 = obstacle[i][1] - obstacle_width / 2
                    y2 = obstacle[i][1] + obstacle_width / 2

                    if x1 > x2:
                        x1, x2 = x2, x1

                    rect_points[0] = (x1, y1)
                    rect_points[1] = (x2, y1)
                    rect_points[2] = (x2, y2)
                    rect_points[3] = (x1, y2)
                else:
                    # https://stackoverflow.com/questions/1250419/finding-points-on-a-line-with-a-given-distance#:~:text=Slope%20m%20is%20just%20the,*%20(m%20*%20s)).
                    m = -1 / ((obstacle[i][1] - obstacle[j][1]) / (obstacle[i][0] - obstacle[j][0]))
                    s = (obstacle_width / 2) / math.sqrt(1 + m * m)

                    rect_points[0] = (obstacle[i][0] - s, obstacle[i][1] - m * s)
                    rect_points[1] = (obstacle[i][0] + s, obstacle[i][1] + m * s)
                    rect_points[2] = (obstacle[j][0] + s, obstacle[j][1] + m * s)
                    rect_points[3] = (obstacle[j][0] - s, obstacle[j][1] - m * s)

                new_obstacles.append(rect_points)

        return new_obstacles

    # iterates through all edges and checks for each edge, if they collide with an obstacle
    # returns only unobstructed edges
    def removeObstructedEdges_parallel(self,edges,obstacles,obstacle_width): 
        # code commented out were in the serial implementation
        # unobstructedEdges = list()
        
        # Arrays holding primitive c++ types for parallelization function used later
        cartesian_product = numpy.array(list(product(range(len(obstacles)), range(len(edges)))))     
        obstacle_array = numpy.array(list(map(lambda x: obstacles[x[0]], cartesian_product)))      
        linepoint1 = numpy.array(list(map(lambda x: edges[x[1]].node1.access_xy(), cartesian_product)))
        linepoint2 = numpy.array(list(map(lambda x: edges[x[1]].node2.access_xy(), cartesian_product)))
        collision = numpy.array([False] * len(cartesian_product))
        collision_check = numpy.array([])

        # Makes calls to the parallelized collision check function in batches of 1024 num_threads
        # For example, if there are 2000 points and 5 obstacles, there will be 10000 pairs of potential collisions to check for
        # We move the data by batches of 1024, num_iter will be 9
        num_threads = 1024
        num_iter = math.ceil(len(cartesian_product) / num_threads)
        for i in range (num_iter):
            start_index = i*1024
            end_index = min(i*1024 + 1024, len(cartesian_product))
            collision_check = numpy.append(collision_check, self.parallel_collision_check(linepoint1[start_index:end_index], linepoint2[start_index:end_index], obstacle_array[start_index:end_index], collision[start_index:end_index]))

        for i in range(len(obstacles) - 1):
            collision_edges = collision_check[i*len(edges) : (i+1)*len(edges)] + collision_check[(i+1) * len(edges) : (i+2)*len(edges)]
      
        # return unobstructedEdges
        return numpy.array(edges)[collision_edges.astype(numpy.bool_) == False].tolist() # for parallel version

    def parallel_collision_check(self, linepoint_1, linepoint_2, rectpoint_1, collision):
      # Type conversion to match with the parameter types of CUDA kernel
      linepoint_1 = linepoint_1.astype(numpy.float32)
      linepoint_2 = linepoint_2.astype(numpy.float32)
      rectpoint_1 = rectpoint_1.astype(numpy.float32)
      collision = collision.astype(numpy.bool_)

      # Memory allocation
      linepoint_1_gpu = cuda.mem_alloc(linepoint_1.nbytes)
      cuda.memcpy_htod(linepoint_1_gpu, linepoint_1)
      linepoint_2_gpu = cuda.mem_alloc(linepoint_2.nbytes)
      cuda.memcpy_htod(linepoint_2_gpu, linepoint_2)
      rectpoint_1_gpu = cuda.mem_alloc(rectpoint_1.nbytes)
      cuda.memcpy_htod(rectpoint_1_gpu, rectpoint_1)
      # rectpoint_2_gpu = cuda.mem_alloc(rectpoint_2.nbytes)
      # cuda.memcpy_htod(rectpoint_2_gpu, rectpoint_2)
      collision_gpu = cuda.mem_alloc(collision.nbytes)
      cuda.memcpy_htod(collision_gpu, collision)    

      # CUDA Kernel
      mod = SourceModule("""
      __global__ void intersects(float *linepoint_1, float *linepoint_2, float *rectpoints, bool *collision)
      {
        // starting indices for linepoints and rectpoints
        int indline = threadIdx.x * 2;
        int indrect = threadIdx.x * 8;

        // Storing all four points of rectangles and both points of line segments in float variables
        float A1x = rectpoints[indrect + 0];
        float A1y = rectpoints[indrect + 1];
        float A2x = rectpoints[indrect + 2];
        float A2y = rectpoints[indrect + 3];
        float A3x = rectpoints[indrect + 4];
        float A3y = rectpoints[indrect + 5];
        float A4x = rectpoints[indrect + 6];
        float A4y = rectpoints[indrect + 7];
        float Ux = linepoint_1[indline + 0];
        float Uy = linepoint_1[indline + 1];
        float Vx = linepoint_2[indline + 0];
        float Vy = linepoint_2[indline + 1];

        //line intersects A1A2
        float denominator = (Ux - Vx) * (A1y - A2y) - (Uy - Vy) * (A1x - A2x);
        if (denominator) {
          float t = ((Ux - A1x) * (A1y - A2y) - (Uy - A1y) * (A1x - A2x)) / denominator;
          float u = -((Ux - Vx) * (Uy - A1y) - (Uy - Vy) * (Ux - A1x)) / denominator;

          if (((t>=0)&&(t<=1)) && ((u>=0)&&(u<=1))) {
            collision[threadIdx.x] = true;
          }
        }

        //line intersects A2A3
        denominator = (Ux - Vx) * (A2y - A3y) - (Uy - Vy) * (A2x - A3x);
        if (denominator) {
          float t = ((Ux - A2x) * (A2y - A3y) - (Uy - A2y) * (A2x - A3x)) / denominator;
          float u = -((Ux - Vx) * (Uy - A2y) - (Uy - Vy) * (Ux - A2x)) / denominator;

          if (((t>=0)&&(t<=1)) && ((u>=0)&&(u<=1))) {
            collision[threadIdx.x] = true;
          }
        }

        //line intersects A3A4
        denominator = (Ux - Vx) * (A3y - A4y) - (Uy - Vy) * (A3x - A4x);
        if (denominator) {
          float t = ((Ux - A3x) * (A3y - A4y) - (Uy - A3y) * (A3x - A4x)) / denominator;
          float u = -((Ux - Vx) * (Uy - A3y) - (Uy - Vy) * (Ux - A3x)) / denominator;

          if (((t>=0)&&(t<=1)) && ((u>=0)&&(u<=1))) {
            collision[threadIdx.x] = true;
          }
        }

        //line intersects A4A1
        denominator = (Ux - Vx) * (A4y - A1y) - (Uy - Vy) * (A4x - A1x);
        if (denominator) {
          float t = ((Ux - A4x) * (A4y - A1y) - (Uy - A4y) * (A4x - A1x)) / denominator;
          float u = -((Ux - Vx) * (Uy - A4y) - (Uy - Vy) * (Ux - A4x)) / denominator;

          if (((t>=0)&&(t<=1)) && ((u>=0)&&(u<=1))) {
            collision[threadIdx.x] = true;
          }
        }
        //if not, falls through and collision boolean remain false
      }
      """)
      # Calls kernel function
      func = mod.get_function("intersects")
      func(linepoint_1_gpu, linepoint_2_gpu, rectpoint_1_gpu, collision_gpu, block=(1024, 1, 1))
      #cuda.Device(0).synchronize()

      # Moving data from device to host
      linepoint_1_checked = numpy.empty_like(linepoint_1)
      cuda.memcpy_dtoh(linepoint_1_checked, linepoint_1_gpu)
      linepoint_2_checked = numpy.empty_like(linepoint_2)
      cuda.memcpy_dtoh(linepoint_2_checked, linepoint_2_gpu)
      rectpoint_1_checked = numpy.empty_like(rectpoint_1)
      cuda.memcpy_dtoh(rectpoint_1_checked, rectpoint_1_gpu)
      # rectpoint_2_checked = numpy.empty_like(rectpoint_2)
      # cuda.memcpy_dtoh(rectpoint_2_checked, rectpoint_2_gpu)
      collision_checked = numpy.empty_like(collision)
      cuda.memcpy_dtoh(collision_checked, collision_gpu)
      

      #cuda.free(linepoint_1_gpu)
      #cuda.free(linepoint_2_gpu)
      #cuda.free(rectpoint_1_gpu)
      #cuda.free(collision_gpu) 

      linepoint_1_gpu.free()
      linepoint_2_gpu.free()
      rectpoint_1_gpu.free()
      collision_gpu.free()
   
      # Return the boolean with collision results
      # For N lines and M obstacles, return an NxM sized 1-D boolean numpy array to indicate collision check result for each (linesegment, obstacle) pair
      return collision_checked


    # perform dijkstra's algorithm from start_node to end_node to find shortest path
    def dijkstra(self, start_node, end_node, edges):
        # Set up data structures
        node_to_distance = {start_node: 0}
        node_to_previous = {start_node: None}
        heap = [(0, start_node)]

        # Perform Dijkstra's algorithm
        while heap:
            (distance, node) = heapq.heappop(heap)
            if node == end_node:
                # We've found the shortest path; build the list of nodes and return it
                path = []
                while node:
                    path.append(node)
                    node = node_to_previous[node]
                return path[::-1]

            for edge in edges:
                if edge.node1 == node:
                    neighbor = edge.node2
                elif edge.node2 == node:
                    neighbor = edge.node1
                else:
                    continue
                
                neighbor_distance = node_to_distance[node] + edge.weight
                if neighbor not in node_to_distance or neighbor_distance < node_to_distance[neighbor]:
                    node_to_distance[neighbor] = neighbor_distance
                    node_to_previous[neighbor] = node
                    heapq.heappush(heap, (neighbor_distance, neighbor))

        # There is no path from start_node to end_node
        return None

    def print_shortest_path(self, shortest_path): 
        if shortest_path:
            print("Shortest path: ")
            for node in shortest_path:
                print(node.access_xy())
        else:
            print("There is no path from start_node to end_node")

# definition of the node class
class Node: 
    def __init__(self, xy, K): 
        self.xy = xy 
        self.k_nearest_neighbors = [None] * K 
        self.u = Util() 

    def access_xy(self):
        return self.xy 

    def __lt__(self, other):
        # Compare two nodes based on their xy coordinates
        return self.xy < other.xy

    def find_nearest_neighbors(self, nodes, K):
        distances = list() 
        for node in nodes: 
            dist = self.u.distance(self.xy,node.xy)
            distances.append((node, dist))
        distances.sort(key=lambda tup: tup[1])

        # ignoring the smallest distance (0) b/c that one is just itself 
        for i in range(1,K+1): 
            self.k_nearest_neighbors[i-1] = distances[i][0]

    def set_nearest_neighbors(self, nodes):
        self.k_nearest_neighbors = nodes


    def display_nearest_neighbors(self):
        print(self.xy)

        for i in range(0,len(self.k_nearest_neighbors)): 
            output_string = "{} near neighbor, location {}, euclidean distance {}".format(i+1, self.k_nearest_neighbors[i].access_xy(),self.u.distance(self.xy, self.k_nearest_neighbors[i].xy))
            print(output_string)
            
# definition of the edge class
class Edge: 
    def __init__(self, node1, node2): 
        self.node1 = node1 
        self.node2 = node2
        self.engaged = True 
        self.u = Util()
        self.weight = self.u.distance(self.node1.access_xy(), self.node2.access_xy())

    def check_equivalent(self, potential_edge): 
        if ((self.node1.access_xy() == potential_edge.node1.access_xy()) and (self.node2.access_xy() == potential_edge.node2.access_xy())) or ((self.node2.access_xy() == potential_edge.node1.access_xy()) and (self.node1.access_xy() == potential_edge.node2.access_xy())):
            return True; 

    def display_edge_info(self): 
        output_string = "node1 @ {}, node 2 @ {}, euclidean distance = {}.".format(self.node1.access_xy(),self.node2.access_xy(),self.u.distance(self.node1.xy, self.node2.xy))        
        print(output_string)

    def get_tuple(self): 
        return (self.node1.xy, self.node2.xy)


# if python says run, then we should run
if __name__ == '__main__':
    main()



Overwriting prm4.py


In [3]:
# from prm import prm
from pycuda.compiler import SourceModule
from prm4 import prm

XDIM = 640
YDIM = 480
w = 100
w2 = 25
h = 150
h2 = 60
X0 = XDIM/5
Y0 = YDIM/3
X1 = 4*XDIM/5
Y1 = 2*YDIM/3
CENTER_SIZE = 20
Obs = {}

Obs[0] = [(XDIM/2, YDIM/7),(XDIM/2, YDIM/3)]
Obs[1] = [(XDIM/2, 6*YDIM/7),(XDIM/2, 2*YDIM/3)]

Obs[2] = [(XDIM/6, YDIM/2),(XDIM/3, YDIM/2)]
Obs[3] = [(5*XDIM/6, YDIM/2),(2*XDIM/3, YDIM/2)]
Obs[4] = [(3*XDIM/6, YDIM/2+20),(2*XDIM/3+24, YDIM/2+60)]
Obs[5] = [(XDIM/2+100, 6*YDIM/7+50),(XDIM/2+140, 2*YDIM/3+30)]

XY_START = (X0+w/2,Y0+3*h/4) # Start in the trap
XY_GOAL = (4*XDIM/5,5*YDIM/6)
XY_GOAL = (X1-w/2,Y1-3*h/4)

print("Serial Parallel Benchmarking ---- ")
game = prm(Obs, XY_START, XY_GOAL, XDIM, YDIM, NUM_SAMPLES = 1000, RUNS = 1)
game.runGame_benchmark()

pygame 2.3.0 (SDL 2.24.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
Serial Parallel Benchmarking ---- 
starting test
total avg time:  2.4558505890008746
sampling avg time:  0.3818737230003535
knn avg time:  0.09202505500070401
check edge collision avg time:  0.30337899799997103
dijkstra's avg time:  1.6653143229996203
